# Create New Groundwater Well Stations on Dendra

Hello, welcome to this script! This script is a tool for creating new Groundwater Wells and their associated datastreams on Dendra. Each well requires a name, lat/long, and elevation (meters). 

If Dendra doesn't have a station to host your data yet, run this cell. It will create a station with your provided name. The station will have 5 datastreams which can be empty if there is no data to fill them. The 5 datastreams are:
- Level from XLE/LEV (1 stream) and XLSX (1 stream) files
- Temperature from XLE/LEV (1 stream) and XLSX (1 stream) files
- Depth to Groundwater (1 stream)

# Load Libraries

In [4]:
import pandas as pd
import numpy as np
import os, glob, linecache, uuid
# import matplotlib.pyplot as plt
import shutil
from scipy.stats import zscore
from copy import deepcopy
import re
import sys
import json

#Import helper functions
from well_data_dendra_helpers import *

uuid_gen = uuid.uuid4()

den_api_lib_path = "C:\\Users\\jinsu.elhance\\Box\\000. Jinsu Elhance\\Github\\dendra-api-client-python"
sys.path.append(den_api_lib_path)
import dendra_api_client as dendra

In [5]:
#Authenticate Dendra API
# If you have a login and the data is not public, you must authenticatte using your Dendra login
dendra.authenticate('jinsu.elhance@tnc.org')

········


In [6]:
#Point to folder where jsons for dendra API can be stored
if not os.path.exists("./DendraJSONs"):
    os.mkdir("./DendraJSONs")
    os.mkdir("./DendraJSONs/Stations")
    os.mkdir("./DendraJSONs/Datastreams")
    os.mkdir("./DendraJSONs/Annotations")
    
dendra_dir = "./DendraJSONs/"

# Provide Information on Wells to Create
Here you can provide a template containing information about the well stations you want to create. For each well you should have the well's name, lat/long coordinates, and elevation in meters. 

In [86]:
wells_to_create = [
    {
        "name": "Escondido 1",
        "lat": "34.504",
        "lon": "-120.479200838299",
        "elevation_m": 159.9,
    },
    {
        "name": "Tinta 6",
        "lat": "34.711",
        "lon": "-120.479",
        "elevation_m": 172.1,
    }
]

## Verify that stations do not already exist on Dendra

In [87]:
#Verify that the wells you are trying to create do not already exist on Dendra.
dendra_wells = dendra.list_stations(orgslug="tnc", query_add={"description":"Groundwater well @ Dangermond Preserve"})

print([dendra_well['name'].replace("Dangermond ","") for dendra_well in dendra_wells])

# dendra_well_metas = [dendra.get_meta_station_by_id(well['_id']) for well in dendra_wells] 

for well in wells_to_create:
    if well['name'] in [dendra_well['name'].replace("Dangermond ", "") for dendra_well in dendra_wells]:
            print(f"Well {well['name']} already exists on Dendra.")

['Alegria', 'Alexander Ramajal', 'Buckhorn 1', 'Buckhorn 2', 'Cojo Canyon', 'Damsite Canyon', 'Diamond Corral', 'East Ramajal', 'Escondido 1', 'Escondido 2', 'Escondido 3', 'Escondido 3B', 'Escondido 4', 'Escondido 5', 'Gaspar 1', 'Gaspar 2', 'Gaucho 1', 'Gaucho 2', 'Jalachichi 1', 'Jalachichi 2', 'Lower Espada', 'Lower Jalama Vaqueros', 'North Ramajal', 'Oaks 1', 'Oaks 2', 'Oaks 3', 'Oaks 3B', 'Oaks 4', 'Oaks 5', 'Old Espada', 'Quail Canyon 1', 'Quarry 1', 'South Ramajal', 'Tinta 1', 'Tinta 10', 'Tinta 11A', 'Tinta 11B', 'Tinta 2', 'Tinta 3', 'Tinta 4', 'Tinta 5', 'Tinta 5B', 'Tinta 6', 'Tinta 7', 'Tinta 8', 'Tinta 9', 'Upper Espada', 'Vaqueros', 'Venadito 1', 'Venadito 2', 'Wood Canyon']
Well Escondido 1 already exists on Dendra.
Well Tinta 6 already exists on Dendra.


# Create Station and Datastreams
If Dendra doesn't have a station to host your data yet, run this cell. It will create a station with your provided name. The station will have 5 datastreams which can be empty if there is no data to fill them. The 5 datastreams are:
- Level from XLE/LEV (1 stream) and XLSX (1 stream) files
- Temperature from XLE/LEV (1 stream) and XLSX (1 stream) files
- Depth to Groundwater (1 stream)

In [88]:
# station_template = deepcopy(station_template) #station template from well_data_dendra_helpers.py
with open(f"{dendra_dir}/Templates/well_station.json") as station_template_raw:
        station_template = deepcopy(json.load(station_template_raw))

json_objects = []

for well in wells_to_create:
    well_station_json = deepcopy(station_template)
    well_station_json["full_name"] = well_station_json["full_name"].replace("WELLNAME", well['name'])
    well_station_json["name"] = well_station_json["name"].replace("WELL NAME", well['name'])
    well_station_json['slug'] = well_station_json['slug'].replace('wellname', well['name'].lower().replace(" ",""))
    well_station_json["geo"] = {
        'type' : 'Point',
        'coordinates' : [float(well['lat']), float(well['lon']), float(well['elevation_m'])]
    }

    # Write the JSON objects to a file
    with open(f"{dendra_dir}/Stations/{well['name']}.station.json", 'w') as json_file:
        json.dump(well_station_json, json_file, indent=4)

In [89]:
#Run this cell to see the stations that will be created
!den meta push-stations --filespec=$dendra_dir/Stations/*.station.json --dry-run --verbose


Will create:   ./DendraJSONs/Stations/Escondido 1.station.json
Will create:   ./DendraJSONs/Stations/Tinta 5B.station.json
Will create:   ./DendraJSONs/Stations/Tinta 6.station.json

Done!


In [90]:
#Run this cell to push those stations to Dendra!
#!den meta push-stations --filespec=$dendra_dir/Stations/*.station.json --verbose

# Create Level and Temperature Datastreams

In [7]:
#First create level and temperature datastreams
datastreams = [("level", "xle_lev"), ("temperature", "xle_lev"), ("temperature", "xlsx"), ("level", "xlsx")]
datastream_template_paths = ["xle_lev_level.json", "xlsx_level.json", "xle_lev_temp.json", "xlsx_temp.json"]
datastream_templates = []

for t in datastream_template_paths: 
    with open(f"{dendra_dir}/Templates/{t}") as template_raw:
        datastream_templates.append(deepcopy(json.load(template_raw)))

In [9]:
dendra_wells = dendra.list_stations(orgslug="tnc", query_add={"description":"Groundwater well @ Dangermond Preserve"})
new_dendra_wells = []

#Only consider newly created wells
for dendra_well in dendra_wells:
    if dendra_well['name'].replace("Dangermond ", "") in [well['name'] for well in wells_to_create]:
        new_dendra_wells.append(dendra_well)
        
print(new_dendra_wells)

#Get metadata from the newly created well stations on dendra
new_dendra_well_metas = [dendra.get_meta_station_by_id(dendra_well['_id']) for dendra_well in new_dendra_wells]

for new_dendra_well in new_dendra_well_metas:
    for i in range(4):
        datastream = datastreams[i]
        datastream_template = deepcopy(datastream_templates[i]) 
        
        #Fill in values on the template to create a new datastream object
        datastream_template['datapoints_config'][0]["params"]["query"]["fc"] = datastream_template['datapoints_config'][0]["params"]["query"]["fc"].replace("WELL", new_dendra_well['name'])
        datastream_template['description'] = datastream_template['description'].replace("STATIONNAME", new_dendra_well['full_name'].replace("Dangermond ", ""))
        datastream_template['datapoints_config_refd'][0]['params']['query']['fc'] = datastream_template['datapoints_config'][0]["params"]["query"]["fc"]
        datastream_template['station_id'] = new_dendra_well['_id']
        
        #Attach coordinates and elevation to each datastream as attributes from their stations.
        datastream_template['geo'] = new_dendra_well['geo']
        datastream_template['attributes'] = {
            'well_elevation_m':new_dendra_well['geo']['coordinates'][2]
        }
        
        #Write JSON objects to be pushed to Dendra.
        with open(f"{dendra_dir}/Datastreams/{new_dendra_well['slug']}.{datastream[0]}.{datastream[1]}.raw.datastream.json", 'w') as json_file:
            json.dump(datastream_template, json_file, indent=4)

[{'_id': '64dbe9ce55264567723ea32c', 'name': 'Dangermond South Ramajal', 'slug': 'dangermond-southramajal'}, {'_id': '654bf9492457b360fd240160', 'name': 'Dangermond Test Well', 'slug': 'dangermond-testwell'}, {'_id': '64dead5cd070872b26d54105', 'name': 'Dangermond Tinta 5B', 'slug': 'dangermond-tinta5b'}]


In [93]:
# !den meta push-datastreams --filespec=$dendra_dir/Datastreams/*raw.datastream.json --dry-run --verbose


Will create:   ./DendraJSONs/Datastreams/dangermond-escondido1.level.xle_lev.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-escondido1.level.xlsx.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-escondido1.temperature.xle_lev.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-escondido1.temperature.xlsx.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-tinta5b.level.xle_lev.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-tinta5b.level.xlsx.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-tinta5b.temperature.xle_lev.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-tinta5b.temperature.xlsx.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-tinta6.level.xle_lev.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams/dangermond-tinta6.level.xlsx.raw.datastream.json
Will create:   ./DendraJSONs/Datastreams

In [94]:
# !den meta push-datastreams --filespec=$dendra_dir/Datastreams/*raw.datastream.json --verbose

# Create Derived Datastreams

In [10]:
with open(f"{dendra_dir}\\Templates\\level_derived.json", encoding="utf-8") as level_template_file:
    level_template_json = json.load(level_template_file)
level_template_json = deepcopy(level_template_json)

with open(f"{dendra_dir}\\Templates\\temperature_derived.json", encoding="utf-8") as temp_template_file:
    temp_template_json = json.load(temp_template_file)
temp_template_json = deepcopy(temp_template_json)

for new_dendra_well in new_dendra_well_metas:
    new_well_datastreams = dendra.list_datastreams_by_station_id(new_dendra_well['_id'])
    well_name_C = new_dendra_well['full_name']
    well_name = new_dendra_well['slug'].replace("dangermond-","")
        
    level_xle, level_xlsx, temp_xle, temp_xlsx = None, None, None, None
    level_json = deepcopy(level_template_json)
    temp_json = deepcopy(temp_template_json)
    
    for datastream in new_well_datastreams: 
        datastream_id = datastream['_id']
        
        if datastream['name'] == "Well Water Level xle/lev":
            level_xle = datastream
        elif datastream['name'] == "Well Water Level xlsx":
            level_xlsx = datastream
        elif datastream['name'] == "Well Water Temperature xle/lev":
            temp_xle = datastream
        elif datastream['name'] == "Well Water Temperature xlsx":
            temp_xlsx = datastream

    #Create Derived Level datastream
    level_json['description'] = f"Derived datastream for {well_name_C} Well Water Level"
    level_json['derived_from_datastream_ids'] = [level_xle['_id'], level_xlsx['_id']]
    level_json['name'] = "Well Water Level"
    level_json['station_id'] = new_dendra_well['_id']
    level_json['geo'] = new_dendra_well['geo']
    level_json['attributes'] = {}
    level_json['attributes']['well_elevation_m'] = new_dendra_well['geo']['coordinates'][2]
    
    with open(f"{dendra_dir}/Datastreams/{well_name}.level.derived.datastream.json", 'w') as json_file:
        json.dump(level_json, json_file, indent=4)
    
    #Create Derived temperature datastream
    temp_json['description'] = f"Derived datastream for {well_name_C} Well Water Temperature"
    temp_json['derived_from_datastream_ids'] = [temp_xle['_id'], temp_xlsx['_id']]
    temp_json['name'] = "Well Water Temperature"
    temp_json['station_id'] = new_dendra_well['_id']
    temp_json['geo'] = new_dendra_well['geo']
    temp_json['attributes'] = {}
    temp_json['attributes']['well_elevation_m'] = new_dendra_well['geo']['coordinates'][2]
    
    with open(f"{dendra_dir}/Datastreams/{well_name}.temperature.derived.datastream.json", 'w') as json_file:
        json.dump(temp_json, json_file, indent=4)

In [11]:
# !den meta push-datastreams --filespec=$dendra_dir/Datastreams/*derived.datastream.json --dry-run --verbose


Will create:   ./DendraJSONs/Datastreams/southramajal.level.derived.datastream.json
Will create:   ./DendraJSONs/Datastreams/southramajal.temperature.derived.datastream.json
Will create:   ./DendraJSONs/Datastreams/testwell.level.derived.datastream.json
Will create:   ./DendraJSONs/Datastreams/testwell.temperature.derived.datastream.json
Will create:   ./DendraJSONs/Datastreams/tinta5b.level.derived.datastream.json
Will create:   ./DendraJSONs/Datastreams/tinta5b.temperature.derived.datastream.json

Done!


In [12]:
# !den meta push-datastreams --filespec=$dendra_dir/Datastreams/*derived.datastream.json --verbose


Created:       ./DendraJSONs/Datastreams/southramajal.level.derived.datastream.json
Created:       ./DendraJSONs/Datastreams/southramajal.temperature.derived.datastream.json
Created:       ./DendraJSONs/Datastreams/testwell.level.derived.datastream.json
Created:       ./DendraJSONs/Datastreams/testwell.temperature.derived.datastream.json
Created:       ./DendraJSONs/Datastreams/tinta5b.level.derived.datastream.json
Created:       ./DendraJSONs/Datastreams/tinta5b.temperature.derived.datastream.json

Done!


# Create Depth to Groundwater Datastream

In [21]:
#This is a tricky one just because each period will need a corresponding sensor depth. I'm not sure if we can make the depth to groundwater stream just run for the entire period. Maybe we need to re-write the datapoint config expression.
with open(f"{dendra_dir}\\Templates\\depthtogroundwater.json", encoding="utf-8") as d2g_template_file:
    d2g_template_json = json.load(d2g_template_file)
d2g_template_json = deepcopy(d2g_template_json)

for new_dendra_well in new_dendra_well_metas:
    new_well_level_datastream = dendra.list_datastreams_by_station_id(new_dendra_well['_id'], query_add={"name":"Well Water Level"})
    d2g_json = deepcopy(d2g_template_json)
    
    #Create Derived Depth to Groundwater Datastream
    d2g_json['derived_from_datastream_ids'] = [new_well_level_datastream[0]['_id']]
    d2g_json['station_id'] = new_dendra_well['_id']
    d2g_json['geo'] = new_dendra_well['geo']
    d2g_json['attributes']['well_elevation_m'] = new_dendra_well['geo']['coordinates'][2]
    
    with open(f"{dendra_dir}/Datastreams/{new_dendra_well['name']}.d2g.derived.datastream.json", 'w') as json_file:
        json.dump(d2g_json, json_file, indent=4)

In [22]:
!den meta push-datastreams --filespec=$dendra_dir/Datastreams/*d2g.derived.datastream.json --dry-run --verbose


Created:       ./DendraJSONs/Datastreams/Dangermond South Ramajal.d2g.derived.datastream.json
Created:       ./DendraJSONs/Datastreams/Dangermond Test Well.d2g.derived.datastream.json
Created:       ./DendraJSONs/Datastreams/Dangermond Tinta 5B.d2g.derived.datastream.json

Done!
